In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
import glob

In [3]:
# pip install konlpy

In [4]:
# pip install tensorflow

In [5]:
# pip install sklearn

## 1. 데이터 불러오기

In [6]:
# 현재 작업 경로 확인
print(os.getcwd())

c:\Users\easya\Workspace\HK_Project\7_자연어처리


### 1-1. "반도체"가 제목에 포함된 네이버 증권 뉴스 데이터

In [7]:
# newstitle 데이터 확인
newstitle = pd.read_csv("../data/Newsfile3.csv", encoding='euc-kr')
newstitle.tail(3)

,date,title
58439,20151019,"[특징주]제주반도체, 美·中 대규모 투자유치에 '上'"
58440,20151019,[반도체] 2016 반도체 투자 감소 예상 - 미래
58441,20151019,"제주반도체, '해외투자유치설' 조회공시 요구"


### 1-2. 휴장일 데이터

In [8]:
# 2015~2016 휴장일 데이터 확인 ()
file1 = pd.read_excel("../data/closedate_2015.xls")
file2 = pd.read_excel("../data/closedate_2016.xls")

In [9]:
# file1 데이터 형식 확인
file1.head(2)

,일자 및 요일,요일구분,비고
0,2015-01-01,목요일,신정
1,2015-02-18,수요일,설날


In [10]:
# file2 데이터 형식 확인
file2.head(2)

,일자 및 요일,요일구분,비고
0,2016-01-01,금요일,신정
1,2016-02-08,월요일,설날


In [11]:
# ./data 에 있는 폴더와 파일경로 확인
files = glob.glob('../data/closedate_*.xls')
files

['../data\\closedate_2015.xls',
 '../data\\closedate_2016.xls',
 '../data\\closedate_2017.xls',
 '../data\\closedate_2018.xls',
 '../data\\closedate_2019.xls',
 '../data\\closedate_2020.xls',
 '../data\\closedate_2021.xls',
 '../data\\closedate_2022.xls']

In [12]:
# files의 첫번째 file만 추출
file = files[0]
file

'../data\\closedate_2015.xls'

In [13]:
# file의 제목만 추출
file.split('\\')[-1]

'closedate_2015.xls'